In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn  as sns
from matplotlib import pyplot as plt

In [ ]:
!pip install catboost

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/data/test_submission.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
train = train.drop(columns='street' , axis = 1)
train = train.drop(columns='id' , axis = 1)
test = test.drop(columns='street' , axis = 1)

In [ ]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
from sklearn.preprocessing import StandardScaler , LabelEncoder , OneHotEncoder

## Разметка данных этажей

In [ ]:
all = { 'подвал, 1' : 1, 'подвал' : 0 ,'цоколь, 1' : 1 , '1,2,антресоль' : 2 , 'цоколь' : 0 , 'тех.этаж (6)' : 6 ,
       'Подвал' : 0 , 'Цоколь' : 0 , 'фактически на уровне 1 этажа' : 1 , '1,2,3' : 3 , '1, подвал' : 1 ,'1,2,3,4' : 4,
       '1,2' : 2 , '1,2,3,4,5' : 5 , '5 , мансарда ' : 5 , '1-й, подвал' : 1 , '1, подвал, антресоль' : 1 , 'мезонин' : 0,
       'подвал, 1-3' : 3 , '1 (Цокольный этаж)' : 1 , '3 , Мансарда (4 эт)' : 3 , 'подвал,1' : 1 , '1 , антресоль' : 1,
       '1-3' : 3 , 'мансарда (4эт)' : 4 , '1, 2.': 2 , 'подвал , 1 ' : 1 , '1, 2' : 2 ,
       '3, Мансарда (4эт)' : 3, '5, мансарда' : 5 , '1, антресоль' : 1, 'подвал, 1,2,3' : 3 ,
         '1 + подвал (без отделки)' : 1 , 'мансарда ' : 2 , '2,3' : 3 , '4, 5' : 5, '1-й, 2-й' : 2,
         '1 этаж, подвал' : 1 , '1, цоколь' : 1 , 'подвал, 1-7, техэтаж' : 7 ,'3 (антресоль)': 3,
         '1, 2, 3' : 3 , 'Цоколь, 1.2(мансарда)' : 2 , 'подвал, 3. 4 этаж' : 4 , 'подвал, 1-4 этаж' : 4,
         'подва, 1.2 этаж' : 2 , '2, 3' : 3 , '-1' : 0 , '1.2' : 2 , '7,8' : 8 , '1 этаж' : 1, '1-й' : 1,
         '3 этаж' : 3 , '4 этаж' : 4 , '5 этаж' : 5 ,'подвал,1,2,3,4,5' : 5 , 'подвал, цоколь, 1 этаж' :1,
         '3, мансарда' : 3 , 'Цоколь, 1,2(мансарда)' : 2 , '3, Мансарда (4 эт)' : 3 , 'мансарда' : 0,
       '-1.0' : 0 , '-2.0' : 0 , ' 1, 2, Антресоль'  : 2 , '3 этаж, мансарда (4 этаж)' : 3 , 
       'цокольный' : 0 , 'подвал, 1 и 4 этаж' : 4 , '5(мансарда)' : 5 , 'технический этаж,5,6' : 6,
       ' 1-2, подвальный' : 2 , '1, 2, 3, мансардный' : 3 , 'подвал, 1, 2, 3' : 3 , 
       '1,2,3, антресоль, технический этаж' : 3 , '3, 4' : 4 , '1-3 этажи, цоколь (188,4 кв.м), подвал (104 кв.м)' : 3,
       '1,2,3,4, подвал' : 4 , '2-й' : 2 , '1, 2 этаж'  : 2 , 'подвал, 1, 2' : 2 , '1-7' : 7 ,  '1 (по док-м цоколь)' : 1,
       '1,2,подвал ' :  2 , 'подвал, 2' : 2 , 'подвал,1,2,3' : 3 , 'подвал,1,2,3' : 3 , '2, 3, 4, тех.этаж' : 4 ,
       'цокольный, 1,2' : 2 , 'Техническое подполье' : 0 , 'цоколь, 1, 2,3,4,5,6' : 6 , '1,2,3 этаж, подвал' : 3
}

In [ ]:
train['floor'] = train['floor'].replace(all)
test['floor'] = test['floor'].replace(all)

In [ ]:
test['floor'].unique()

array(['1.0', 0, nan, 3, '2', '5.0', '4', '1', '5', 2, '2.0', '4.0', 4,
       '3', '3.0', '11.0', '9.0', '12.0', '0.0', 6, '6', '6.0', '13', 1,
       '7', '1,2 ', '14', '9', '8', '3,4', '10.0', 5, '8.0', '7.0', 7,
       '10', '13.0'], dtype=object)

In [ ]:
corr = train.corr()
corr[abs(corr['per_square_meter_price']) > 0.1]['per_square_meter_price']

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
train['city'] = le.fit_transform(train['city'])
test['city'] = oh.fit_transform(test['city'])

In [ ]:
train.head()

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,per_square_meter_price,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,0,0,0,0,2,4,6,3.293470,Пермь,1055397.0,0.027732,3,6,17,34,0,0,1,1,0,0,1,2,2,3,4,0,0,1,0,0,0,0,1,2,0,1,2,4,4,11,20,28,269.024986,3.368385,0,0,0,0.002864,4,13,21,139937.500000,136,49,2503.0,765.0,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.0,S27289,2020-01-05,10,0
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,0,0,1,0,2,2,6,43.950989,Орехово-Зуево,120184.0,0.089441,0,31,50,57,0,1,2,3,0,0,1,2,1,1,3,2,4,6,2,2,2,1,1,2,0,1,2,3,1,12,20,29,102.455451,1.476600,0,0,0,0.154661,4,10,11,60410.714286,146,37,1336.0,514.0,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.0,S17052,2020-01-05,10,0
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,0,1,1,0,3,6,11,2.676293,Ярославль,603961.0,0.200995,0,15,29,53,0,1,2,2,0,0,5,9,0,1,3,0,0,0,0,0,1,2,3,6,0,1,6,9,1,16,37,80,243.361937,1.455127,0,0,0,0.118275,9,13,21,45164.761264,105,27,1883.0,573.0,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.4,S16913,2020-01-05,110,0
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,0,0,1,0,0,1,4,15.618563,Новокузнецк,552105.0,0.861400,0,0,0,5,0,0,0,0,0,0,0,1,0,0,3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,4,11,295.633502,9.400855,0,0,0,1.036523,0,0,3,28805.263158,75,2,1801.0,54.0,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.0,S10148,2020-01-05,110,0
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,1,1,1,0,8,14,26,9.995325,Химки,232066.0,0.236744,0,14,40,78,0,0,0,0,0,0,3,9,2,7,14,0,2,2,1,1,3,3,4,9,0,8,8,12,0,8,33,89,1.110429,1.235503,0,0,0,0.235032,10,32,62,13222.591362,144,38,3090.0,619.0,7.263889,5.684211,1963.229167,1960.500000,Москва,60.2,S1338,2020-01-05,10,0


In [ ]:
train['region'] = le.fit_transform(train['region'])
test['region'] = le.fit_transform(test['region'])
train['osm_city_nearest_name'] = le.fit_transform(train['osm_city_nearest_name'])
test['osm_city_nearest_name'] = le.fit_transform(test['osm_city_nearest_name'])

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

In [ ]:
median = train['floor'].median()
train['floor'].fillna(median, inplace=True)

In [ ]:
train.isnull().sum()

city            0
floor           0
id              0
lat             0
lng             0
               ..
region          0
total_square    0
date            0
realty_type     0
price_type      0
Length: 76, dtype: int64

In [ ]:
data_approved = train[train.price_type == 1]
data_not_approved = train[train.price_type == 0]

In [ ]:
data_approved

In [ ]:
X = data_approved.drop(['per_square_meter_price'], axis=1)
y = data_approved.per_square_meter_price

In [ ]:
!pip install optuna

In [ ]:
import optuna

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X , y, test_size = 0.2 , random_state = 42)

In [ ]:
cat_cols = ['city' , 'region' , 'osm_city_nearest_name']

In [ ]:
model = CatBoostRegressor(cat_features= cat_cols)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [ ]:
mean_absolute_error(y_test,y_pred)

20586.264471700688

In [ ]:
mean_absolute_error(y_test,y_lgb)

31191.83597193549

In [ ]:
cat = CatBoostRegressor(**study_cat.best_trial.params)

In [ ]:
from sklearn.ensemble import VotingRegressor

In [ ]:
vot = VotingRegressor(estimators=[ ('cat', cat), ('lgb', lgb)],n_jobs= -1)
vot.fit(X_train , y_train)
y_vot = vot.predict(X_test)

In [ ]:
vot.score(X_train , y_train) , vot.score(X_test , y_test)

(0.8655595088474123, 0.8106428863548287)

In [ ]:
key_cols =  ['id' , 'floor' , 'city' , 'region']

In [ ]:
y_final = model.predict(test.drop(key_cols,axis = 1))

In [ ]:
test['per_square_meter_price'] = pred_test

In [ ]:
test[['id' , 'per_square_meter_price']].to_csv('sub6.csv' , header = True , index = False)